# 🔍 LangSmith: O Sherlock Holmes das suas Aplicações LangChain!

**Módulo 17 - Curso LangChain v0.3**

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_01.png)

Bora para o último módulo galera! Tá, já aprendemos a criar, agora vamos aprender a **MONITORAR** e **DEBUGGAR** nossas aplicações!

**Pedro Nunes Guth** 🚀

## 🤔 Tá, mas o que é o LangSmith?

Imagina que você tem um restaurante (sua aplicação LangChain). Você sabe fazer a comida, mas como saber:
- Quais pratos os clientes mais gostaram?
- Onde a cozinha está demorada?
- Qual garçom está errando os pedidos?
- Quanto está custando cada prato?

**LangSmith é seu sistema de monitoramento completo!**

### O que o LangSmith faz:
1. **Tracing**: Acompanha cada passo da sua chain
2. **Logging**: Registra tudo que acontece
3. **Evaluation**: Testa se sua IA está funcionando bem
4. **Monitoring**: Alerta quando algo vai mal
5. **Debugging**: Mostra onde está o problema

**Dica!** É tipo um Google Analytics para suas aplicações de IA!

In [ ]:
# Vamos instalar o que precisamos
!pip install langsmith langchain-google-genai python-dotenv matplotlib seaborn -q

print("📦 Pacotes instalados!")
print("🔑 Agora você vai precisar de uma conta no LangSmith (smith.langchain.com)")

In [ ]:
# Imports necessários
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import json

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Bibliotecas carregadas com sucesso!")

## ⚙️ Configurando o LangSmith

Primeiro, você precisa:
1. Criar uma conta em [smith.langchain.com](https://smith.langchain.com)
2. Pegar sua API Key
3. Configurar as variáveis de ambiente

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_02.png)

**Dica!** É de graça para uso pessoal e pequenos projetos!

In [ ]:
# Carregando as variáveis de ambiente
load_dotenv()

# Configurações do LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Habilita o tracing
os.environ["LANGCHAIN_PROJECT"] = "meu-projeto-langchain"  # Nome do projeto

# Suas chaves (substitua pelos valores reais)
# os.environ["LANGCHAIN_API_KEY"] = "sua_langsmith_api_key_aqui"
# os.environ["GOOGLE_API_KEY"] = "sua_google_api_key_aqui"

print("🔧 Configurações do LangSmith definidas!")
print(f"📋 Projeto: {os.environ.get('LANGCHAIN_PROJECT')}")
print(f"🔍 Tracing: {os.environ.get('LANGCHAIN_TRACING_V2')}")

## 🏗️ Criando uma Aplicação para Monitorar

Vamos criar uma aplicação simples usando os conceitos que já aprendemos no curso:
- **ChatModel** (Módulo 2)
- **PromptTemplate** (Módulo 4)
- **Chains** (Módulo 6)
- **RAG** (Módulo 10)

### Fluxo da Nossa Aplicação:
```mermaid
graph TD
    A[Pergunta do Usuário] --> B[Chain de Análise]
    B --> C[Modelo Gemini]
    C --> D[Resposta Processada]
    D --> E[LangSmith Monitoring]
    E --> F[Dashboard]
```

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Criando o modelo (lembrando do Módulo 2)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.7
)

# Criando o prompt template (lembrando do Módulo 4)
prompt_template = PromptTemplate(
    template="""
    Você é um assistente especializado em análise de sentimentos.
    
    Analise o texto abaixo e forneça:
    1. Sentimento (Positivo/Neutro/Negativo)
    2. Confiança (0-100%)
    3. Palavras-chave que influenciaram sua decisão
    
    Texto: {texto}
    
    Resposta:
    """,
    input_variables=["texto"]
)

print("🤖 Modelo e template criados!")

In [ ]:
# Criando a chain (lembrando do Módulo 6)
chain_analise_sentimento = (
    {"texto": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

print("⛓️ Chain criada!")
print("🎯 Agora todas as execuções serão monitoradas pelo LangSmith")

## 🚀 Testando com Monitoramento Ativo

Agora vamos executar nossa chain várias vezes para gerar dados no LangSmith!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_03.png)

**Dica!** Cada execução será automaticamente logada no LangSmith!

In [ ]:
# Lista de textos para testar nossa aplicação
textos_teste = [
    "Adorei este produto! Superou minhas expectativas!",
    "O atendimento foi péssimo, não recomendo.",
    "O produto chegou no prazo esperado.",
    "Incrível! Melhor compra que já fiz na vida!",
    "Tive problemas com a entrega, mas o produto é bom.",
    "Funcionou conforme descrito na embalagem.",
    "Que decepção! Nada do que foi prometido.",
    "Excelente qualidade, vale cada centavo!"
]

print(f"📝 Temos {len(textos_teste)} textos para analisar")
print("🔄 Cada execução será trackeada pelo LangSmith!")

In [ ]:
# Executando as análises (isso gerará traces no LangSmith)
resultados = []

for i, texto in enumerate(textos_teste):
    print(f"\n🔍 Analisando texto {i+1}/{len(textos_teste)}...")
    print(f"📄 Texto: {texto[:50]}...")
    
    try:
        # Esta execução será automaticamente trackeada!
        resultado = chain_analise_sentimento.invoke(texto)
        resultados.append({
            "texto": texto,
            "resultado": resultado,
            "status": "sucesso"
        })
        print("✅ Análise concluída!")
        
    except Exception as e:
        print(f"❌ Erro na análise: {e}")
        resultados.append({
            "texto": texto,
            "resultado": None,
            "status": "erro",
            "erro": str(e)
        })

print(f"\n🎯 Processamento concluído! {len(resultados)} análises realizadas")
print("📊 Verifique seu dashboard do LangSmith para ver os traces!")

## 📊 Análise Local dos Resultados

Enquanto o LangSmith coleta os dados detalhados, vamos fazer uma análise local dos nossos resultados!

### Métricas que o LangSmith coleta automaticamente:
- **Latência**: Tempo de resposta
- **Tokens**: Consumo de tokens de entrada e saída
- **Custo**: Quanto cada execução custou
- **Erros**: Taxa de falha
- **Traces**: Fluxo completo da execução

**Dica!** No dashboard você vê tudo isso em tempo real!

In [ ]:
# Analisando os resultados localmente
sucessos = [r for r in resultados if r["status"] == "sucesso"]
erros = [r for r in resultados if r["status"] == "erro"]

print(f"📈 Relatório de Execução:")
print(f"✅ Sucessos: {len(sucessos)} ({len(sucessos)/len(resultados)*100:.1f}%)")
print(f"❌ Erros: {len(erros)} ({len(erros)/len(resultados)*100:.1f}%)")

# Simulando métricas de performance
latencias_simuladas = np.random.normal(1.2, 0.3, len(sucessos))  # Segundos
tokens_simulados = np.random.randint(50, 200, len(sucessos))     # Tokens

print(f"\n⏱️ Performance Simulada:")
print(f"📊 Latência média: {np.mean(latencias_simuladas):.2f}s")
print(f"🎯 Tokens médios: {np.mean(tokens_simulados):.0f} tokens")

In [ ]:
# Criando visualizações como as que você veria no LangSmith
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Taxa de Sucesso
labels = ['Sucessos', 'Erros']
sizes = [len(sucessos), len(erros)]
colors = ['#2ecc71', '#e74c3c']
ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Taxa de Sucesso das Execuções', fontsize=14, pad=20)

# Gráfico 2: Distribuição de Latência
ax2.hist(latencias_simuladas, bins=10, color='#3498db', alpha=0.7, edgecolor='black')
ax2.set_title('Distribuição de Latência', fontsize=14, pad=20)
ax2.set_xlabel('Tempo (segundos)')
ax2.set_ylabel('Frequência')
ax2.axvline(np.mean(latencias_simuladas), color='red', linestyle='--', label=f'Média: {np.mean(latencias_simuladas):.2f}s')
ax2.legend()

# Gráfico 3: Consumo de Tokens
ax3.bar(range(len(tokens_simulados)), tokens_simulados, color='#9b59b6', alpha=0.7)
ax3.set_title('Consumo de Tokens por Execução', fontsize=14, pad=20)
ax3.set_xlabel('Execução')
ax3.set_ylabel('Tokens Utilizados')
ax3.axhline(np.mean(tokens_simulados), color='red', linestyle='--', label=f'Média: {np.mean(tokens_simulados):.0f}')
ax3.legend()

# Gráfico 4: Timeline de Execuções
horarios = [datetime.now() - timedelta(minutes=x*2) for x in range(len(sucessos))]
ax4.plot(horarios, latencias_simuladas, 'o-', color='#e67e22', linewidth=2, markersize=6)
ax4.set_title('Timeline de Performance', fontsize=14, pad=20)
ax4.set_xlabel('Tempo')
ax4.set_ylabel('Latência (s)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 Esses são os tipos de gráficos que você vê no LangSmith dashboard!")

## 🔍 Tracing Detalhado: O Que Acontece Sob o Capô

O **Tracing** é a funcionalidade mais poderosa do LangSmith. É como ter raio-X da sua aplicação!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_04.png)

### O que o Trace mostra:
1. **Input**: O que entrou
2. **Steps**: Cada passo da chain
3. **Output**: O que saiu
4. **Timing**: Quanto tempo cada parte demorou
5. **Metadata**: Informações extras (tokens, modelo usado, etc.)

**Dica!** É tipo o "Inspect Element" do navegador, mas para IA!

In [ ]:
# Simulando um trace detalhado (como você veria no LangSmith)
exemplo_trace = {
    "run_id": "abc-123-def-456",
    "name": "chain_analise_sentimento",
    "start_time": "2024-01-15T10:30:00Z",
    "end_time": "2024-01-15T10:30:02Z",
    "duration_ms": 2000,
    "inputs": {
        "texto": "Adorei este produto! Superou minhas expectativas!"
    },
    "outputs": {
        "content": "Sentimento: Positivo\nConfiança: 95%\nPalavras-chave: adorei, superou, expectativas"
    },
    "metadata": {
        "model": "gemini-2.0-flash-exp",
        "tokens_input": 45,
        "tokens_output": 23,
        "cost_usd": 0.0012
    },
    "children": [
        {
            "name": "PromptTemplate",
            "duration_ms": 5,
            "type": "prompt"
        },
        {
            "name": "ChatGoogleGenerativeAI",
            "duration_ms": 1800,
            "type": "llm"
        },
        {
            "name": "StrOutputParser",
            "duration_ms": 2,
            "type": "parser"
        }
    ]
}

# Exibindo o trace de forma organizada
print("🔍 EXEMPLO DE TRACE DETALHADO:")
print("=" * 50)
print(f"📋 Run ID: {exemplo_trace['run_id']}")
print(f"⏱️  Duração Total: {exemplo_trace['duration_ms']}ms")
print(f"💰 Custo: ${exemplo_trace['metadata']['cost_usd']:.4f}")
print(f"🎯 Tokens: {exemplo_trace['metadata']['tokens_input']} → {exemplo_trace['metadata']['tokens_output']}")

print("\n📝 INPUT:")
print(f"   {exemplo_trace['inputs']['texto']}")

print("\n📤 OUTPUT:")
print(f"   {exemplo_trace['outputs']['content']}")

print("\n🔧 STEPS EXECUTADOS:")
for i, step in enumerate(exemplo_trace['children']):
    print(f"   {i+1}. {step['name']} ({step['duration_ms']}ms)")

## 🧪 Evaluation: Testando se sua IA Está Boa

Uma das funcionalidades mais **liiinda** do LangSmith é a **Evaluation**!

É tipo ter um professor que corrige as respostas da sua IA automaticamente!

### Tipos de Evaluation:
1. **Manual**: Você mesmo avalia
2. **Automática**: Outra IA avalia
3. **Métricas**: BLEU, ROUGE, etc.
4. **Custom**: Suas próprias regras

**Dica!** Use evaluation para comparar diferentes versões do seu prompt!

In [ ]:
# Simulando uma evaluation automática
def avaliar_sentimento(texto_original, resposta_ia):
    """
    Função que simula como o LangSmith avaliaria nossa análise de sentimento
    """
    # Palavras que indicam sentimento conhecido
    palavras_positivas = ['adorei', 'excelente', 'incrível', 'superou', 'melhor']
    palavras_negativas = ['péssimo', 'decepção', 'problemas', 'ruim']
    
    texto_lower = texto_original.lower()
    resposta_lower = resposta_ia.lower()
    
    # Determinar sentimento esperado
    if any(palavra in texto_lower for palavra in palavras_positivas):
        sentimento_esperado = 'positivo'
    elif any(palavra in texto_lower for palavra in palavras_negativas):
        sentimento_esperado = 'negativo'
    else:
        sentimento_esperado = 'neutro'
    
    # Verificar se a IA acertou
    acertou = sentimento_esperado in resposta_lower
    
    # Calcular score (simulado)
    score = 1.0 if acertou else 0.0
    
    return {
        'esperado': sentimento_esperado,
        'acertou': acertou,
        'score': score,
        'confianca': np.random.uniform(0.7, 0.95) if acertou else np.random.uniform(0.3, 0.6)
    }

print("🧪 Função de evaluation criada!")
print("📊 Agora vamos avaliar nossos resultados...")

In [ ]:
# Avaliando nossos resultados
avaliacoes = []

print("🔍 AVALIAÇÃO DOS RESULTADOS:")
print("=" * 60)

for resultado in sucessos:
    avaliacao = avaliar_sentimento(resultado['texto'], resultado['resultado'])
    avaliacoes.append(avaliacao)
    
    status = "✅ CORRETO" if avaliacao['acertou'] else "❌ INCORRETO"
    print(f"{status} | Score: {avaliacao['score']:.1f} | Confiança: {avaliacao['confianca']:.2f}")
    print(f"   Texto: {resultado['texto'][:40]}...")
    print(f"   Esperado: {avaliacao['esperado'].title()}")
    print()

# Calculando métricas finais
accuracy = np.mean([a['score'] for a in avaliacoes])
confianca_media = np.mean([a['confianca'] for a in avaliacoes])

print(f"📊 MÉTRICAS FINAIS:")
print(f"🎯 Accuracy: {accuracy:.2%}")
print(f"💪 Confiança Média: {confianca_media:.2%}")
print(f"📈 Total Avaliado: {len(avaliacoes)} execuções")

## 🚨 Monitoring e Alertas

O LangSmith pode te avisar quando algo vai mal! É tipo ter um WhatsApp que te manda mensagem quando sua IA tá com problema.

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_05.png)

### Tipos de Alertas:
- **Alta Latência**: Quando demora muito
- **Taxa de Erro**: Quando falha muito
- **Custo Elevado**: Quando gasta muito
- **Uso Anômalo**: Quando algo estranho acontece

**Dica!** Configure alertas antes de colocar em produção!

In [ ]:
# Simulando um sistema de monitoramento
class MonitorLangSmith:
    def __init__(self):
        self.limites = {
            'latencia_max': 3.0,  # segundos
            'erro_rate_max': 0.05,  # 5%
            'custo_diario_max': 10.0,  # USD
            'tokens_por_min_max': 1000
        }
        self.alertas = []
    
    def verificar_metricas(self, latencias, erros, custos, tokens):
        # Verificar latência
        latencia_media = np.mean(latencias)
        if latencia_media > self.limites['latencia_max']:
            self.alertas.append(f"🚨 ALERTA: Latência alta ({latencia_media:.2f}s)")
        
        # Verificar taxa de erro
        erro_rate = len(erros) / (len(sucessos) + len(erros))
        if erro_rate > self.limites['erro_rate_max']:
            self.alertas.append(f"🚨 ALERTA: Taxa de erro alta ({erro_rate:.2%})")
        
        # Verificar custo
        custo_total = sum(custos)
        if custo_total > self.limites['custo_diario_max']:
            self.alertas.append(f"💰 ALERTA: Custo elevado (${custo_total:.2f})")
        
        # Verificar tokens por minuto
        tokens_por_min = sum(tokens) / 10  # Simulando 10 minutos
        if tokens_por_min > self.limites['tokens_por_min_max']:
            self.alertas.append(f"📊 ALERTA: Uso alto de tokens ({tokens_por_min:.0f}/min)")
    
    def gerar_relatorio(self):
        if self.alertas:
            print("🚨 ALERTAS DETECTADOS:")
            for alerta in self.alertas:
                print(f"   {alerta}")
        else:
            print("✅ Tudo funcionando perfeitamente!")

# Testando o monitor
monitor = MonitorLangSmith()

# Dados simulados
custos_simulados = np.random.uniform(0.001, 0.005, len(sucessos))

monitor.verificar_metricas(
    latencias_simuladas,
    erros,
    custos_simulados,
    tokens_simulados
)

monitor.gerar_relatorio()

print(f"\n📊 Métricas Atuais:")
print(f"⏱️  Latência média: {np.mean(latencias_simuladas):.2f}s")
print(f"❌ Taxa de erro: {len(erros)/(len(sucessos)+len(erros)):.2%}")
print(f"💰 Custo total: ${sum(custos_simulados):.4f}")
print(f"🎯 Tokens/min: {sum(tokens_simulados)/10:.0f}")

## 🔧 Debugging com LangSmith

Quando algo dá errado (e sempre dá 😅), o LangSmith é seu melhor amigo!

### Como debuggar:
1. **Encontre o erro no trace**
2. **Veja o input que causou**
3. **Analise cada step**
4. **Teste correções**
5. **Compare resultados**

**Dica!** Use os filtros do dashboard para encontrar erros rapidamente!

In [ ]:
# Simulando um processo de debugging
def debug_execucao_falhada():
    print("🐛 SIMULAÇÃO DE DEBUGGING")
    print("=" * 40)
    
    # Simulando um erro comum
    erro_exemplo = {
        "run_id": "erro-123-abc",
        "input": "Analyze this sentiment: [TEXTO MUITO LONGO COM 50000 CARACTERES...]",
        "error": "TokenLimitExceededError: Input too long (50000 > 32000 tokens)",
        "timestamp": "2024-01-15T15:30:00Z",
        "step_failed": "ChatGoogleGenerativeAI"
    }
    
    print(f"❌ ERRO DETECTADO:")
    print(f"   ID: {erro_exemplo['run_id']}")
    print(f"   Erro: {erro_exemplo['error']}")
    print(f"   Step que falhou: {erro_exemplo['step_failed']}")
    
    print(f"\n🔍 ANÁLISE DO PROBLEMA:")
    print(f"   ➤ Input muito grande (>32k tokens)")
    print(f"   ➤ Modelo tem limite de contexto")
    print(f"   ➤ Precisa de text splitter")
    
    print(f"\n💡 SOLUÇÕES PROPOSTAS:")
    print(f"   1. Implementar text splitter (Módulo 8)")
    print(f"   2. Resumir texto antes de analisar")
    print(f"   3. Usar modelo com contexto maior")
    print(f"   4. Processar em chunks menores")
    
    return erro_exemplo

erro = debug_execucao_falhada()

## 📈 Comparando Versões e A/B Testing

Uma das funcionalidades mais **poderosas** do LangSmith é comparar diferentes versões!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_06.png)

### O que você pode comparar:
- **Diferentes prompts**
- **Modelos diferentes**
- **Versões da sua aplicação**
- **Parâmetros (temperatura, etc.)**

**Dica!** Sempre teste antes de fazer deploy!

In [ ]:
# Simulando A/B Testing entre dois prompts
prompt_v1 = """
Analise o sentimento do texto: {texto}
Responda apenas: Positivo, Neutro ou Negativo
"""

prompt_v2 = """
Você é um especialista em análise de sentimentos.
Analise cuidadosamente o texto abaixo e determine:
1. Sentimento (Positivo/Neutro/Negativo)
2. Confiança (0-100%)
3. Justificativa

Texto: {texto}
"""

# Simulando métricas para cada versão
metricas_v1 = {
    "accuracy": 0.75,
    "latencia_media": 0.8,
    "tokens_medio": 25,
    "custo_medio": 0.0008,
    "satisfacao_usuario": 3.2
}

metricas_v2 = {
    "accuracy": 0.92,
    "latencia_media": 1.4,
    "tokens_medio": 65,
    "custo_medio": 0.0021,
    "satisfacao_usuario": 4.6
}

print("🆚 COMPARAÇÃO A/B TESTING")
print("=" * 50)
print(f"📊 Prompt V1 (Simples):")
for metrica, valor in metricas_v1.items():
    print(f"   {metrica}: {valor}")

print(f"\n📊 Prompt V2 (Detalhado):")
for metrica, valor in metricas_v2.items():
    print(f"   {metrica}: {valor}")

print(f"\n🏆 VENCEDOR: Prompt V2")
print(f"   ✅ Maior accuracy (+17%)")
print(f"   ✅ Melhor satisfação (+44%)")
print(f"   ❌ Maior latência (+75%)")
print(f"   ❌ Maior custo (+162%)")

In [ ]:
# Visualizando a comparação
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metricas = ['accuracy', 'latencia_media', 'custo_medio', 'satisfacao_usuario']
titulos = ['Accuracy', 'Latência Média (s)', 'Custo Médio ($)', 'Satisfação do Usuário']
cores = ['#2ecc71', '#e74c3c', '#f39c12', '#9b59b6']

for i, (metrica, titulo, cor) in enumerate(zip(metricas, titulos, cores)):
    ax = axes[i//2, i%2]
    
    v1_valor = metricas_v1[metrica]
    v2_valor = metricas_v2[metrica]
    
    ax.bar(['Prompt V1', 'Prompt V2'], [v1_valor, v2_valor], 
           color=[cor, cor], alpha=[0.6, 1.0], edgecolor='black')
    
    ax.set_title(titulo, fontsize=12, pad=15)
    ax.set_ylabel('Valor')
    
    # Adicionar valores nas barras
    for j, valor in enumerate([v1_valor, v2_valor]):
        ax.text(j, valor + valor*0.05, f'{valor}', ha='center', fontweight='bold')

plt.suptitle('Comparação A/B Testing: Prompt V1 vs V2', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("📊 Esse tipo de comparação é automática no LangSmith!")

## 💰 Controle de Custos e Otimização

LangSmith te ajuda a não quebrar o orçamento! É tipo ter um contador sempre de olho nos gastos.

### Métricas de Custo:
- **Custo por execução**
- **Custo por usuário**
- **Custo por dia/mês**
- **Projeções de gasto**
- **Otimizações sugeridas**

**Dica!** Configure limites de gasto para não ter surpresas!

In [ ]:
# Simulando análise de custos
def analisar_custos():
    # Simulando dados de uma semana
    dias = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom']
    execucoes_por_dia = [150, 200, 180, 220, 250, 100, 80]
    custo_por_execucao = [0.0015, 0.0012, 0.0018, 0.0011, 0.0020, 0.0014, 0.0016]
    
    custos_diarios = [exec * custo for exec, custo in zip(execucoes_por_dia, custo_por_execucao)]
    
    # Análise
    custo_total_semana = sum(custos_diarios)
    custo_medio_dia = np.mean(custos_diarios)
    projecao_mensal = custo_total_semana * 4.33  # 4.33 semanas por mês
    
    print("💰 ANÁLISE DE CUSTOS - ÚLTIMA SEMANA")
    print("=" * 45)
    
    for dia, execucoes, custo_dia in zip(dias, execucoes_por_dia, custos_diarios):
        print(f"{dia}: {execucoes:3d} execuções = ${custo_dia:.4f}")
    
    print(f"\n📊 RESUMO:")
    print(f"   Total da semana: ${custo_total_semana:.4f}")
    print(f"   Média por dia: ${custo_medio_dia:.4f}")
    print(f"   Projeção mensal: ${projecao_mensal:.2f}")
    
    # Alertas e sugestões
    if projecao_mensal > 50:
        print(f"\n🚨 ALERTA: Projeção alta para o mês!")
        print(f"💡 Sugestões:")
        print(f"   - Otimizar prompts (reduzir tokens)")
        print(f"   - Usar cache para respostas repetidas")
        print(f"   - Implementar rate limiting")
    else:
        print(f"\n✅ Custos sob controle!")
    
    return dias, execucoes_por_dia, custos_diarios

dias, execucoes, custos = analisar_custos()

In [ ]:
# Visualização dos custos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Custos por dia
bars1 = ax1.bar(dias, custos, color='#e74c3c', alpha=0.7, edgecolor='black')
ax1.set_title('Custos Diários', fontsize=14, pad=15)
ax1.set_ylabel('Custo ($)')
ax1.set_xlabel('Dia da Semana')

# Adicionar valores nas barras
for bar, custo in zip(bars1, custos):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.05,
             f'${custo:.3f}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Execuções vs Custo
ax2.scatter(execucoes, custos, c=range(len(dias)), s=100, alpha=0.7, 
           cmap='viridis', edgecolors='black')

# Adicionar linha de tendência
z = np.polyfit(execucoes, custos, 1)
p = np.poly1d(z)
ax2.plot(execucoes, p(execucoes), "r--", alpha=0.8, linewidth=2)

ax2.set_title('Execuções vs Custo', fontsize=14, pad=15)
ax2.set_xlabel('Número de Execuções')
ax2.set_ylabel('Custo ($)')

# Adicionar labels dos dias
for i, (exec_count, custo, dia) in enumerate(zip(execucoes, custos, dias)):
    ax2.annotate(dia, (exec_count, custo), xytext=(5, 5), 
                textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

print("📊 No LangSmith você tem esses gráficos atualizados em tempo real!")

## 🏋️‍♂️ Exercício Prático: Implementando Monitoramento Completo

Bora colocar a mão na massa! Vamos criar um sistema de monitoramento completo para uma aplicação RAG!

**Desafio**: Implementar monitoramento para um sistema que responde perguntas sobre documentos.

In [ ]:
# EXERCÍCIO: Complete o código abaixo
# Vamos simular uma aplicação RAG com monitoramento

class SistemaRAGMonitorado:
    def __init__(self):
        self.metricas = {
            'total_queries': 0,
            'sucessos': 0,
            'erros': 0,
            'tempo_total': 0,
            'custo_total': 0,
            'historico': []
        }
    
    def processar_query(self, pergunta, documentos):
        """Simula processamento de uma query RAG"""
        import time
        import random
        
        inicio = time.time()
        self.metricas['total_queries'] += 1
        
        try:
            # Simula processamento RAG
            time.sleep(random.uniform(0.1, 0.5))  # Simula tempo de processamento
            
            # TODO: Adicione aqui a lógica de processamento real
            # Dica: Use os conceitos do Módulo 10 (RAG)
            
            resposta = f"Baseado nos documentos, a resposta para '{pergunta[:30]}...' é: [RESPOSTA SIMULADA]"
            
            # Métricas de sucesso
            tempo_execucao = time.time() - inicio
            custo_estimado = random.uniform(0.002, 0.008)
            
            self.metricas['sucessos'] += 1
            self.metricas['tempo_total'] += tempo_execucao
            self.metricas['custo_total'] += custo_estimado
            
            # TODO: Adicione mais métricas aqui
            # - Relevância da resposta
            # - Satisfação do usuário
            # - Documentos utilizados
            
            self.metricas['historico'].append({
                'pergunta': pergunta,
                'resposta': resposta,
                'tempo': tempo_execucao,
                'custo': custo_estimado,
                'status': 'sucesso',
                'timestamp': datetime.now()
            })
            
            return resposta
            
        except Exception as e:
            # Tratamento de erro
            self.metricas['erros'] += 1
            
            self.metricas['historico'].append({
                'pergunta': pergunta,
                'resposta': None,
                'tempo': time.time() - inicio,
                'custo': 0,
                'status': 'erro',
                'erro': str(e),
                'timestamp': datetime.now()
            })
            
            raise e
    
    def gerar_dashboard(self):
        """Gera um dashboard de métricas"""
        print("📊 DASHBOARD DE MONITORAMENTO RAG")
        print("=" * 50)
        
        # TODO: Complete as métricas
        taxa_sucesso = self.metricas['sucessos'] / max(self.metricas['total_queries'], 1)
        tempo_medio = self.metricas['tempo_total'] / max(self.metricas['sucessos'], 1)
        custo_medio = self.metricas['custo_total'] / max(self.metricas['sucessos'], 1)
        
        print(f"📈 Total de Queries: {self.metricas['total_queries']}")
        print(f"✅ Taxa de Sucesso: {taxa_sucesso:.2%}")
        print(f"⏱️  Tempo Médio: {tempo_medio:.2f}s")
        print(f"💰 Custo Médio: ${custo_medio:.4f}")
        print(f"💸 Custo Total: ${self.metricas['custo_total']:.4f}")
        
        # TODO: Adicione mais métricas aqui
        
        return {
            'taxa_sucesso': taxa_sucesso,
            'tempo_medio': tempo_medio,
            'custo_medio': custo_medio
        }

# Testando o sistema
sistema = SistemaRAGMonitorado()
print("🚀 Sistema RAG com monitoramento criado!")
print("💡 Agora complete as partes marcadas com TODO")

In [ ]:
# Testando o sistema com queries simuladas
queries_teste = [
    "Qual é a política de devolução?",
    "Como funciona o sistema de pagamento?",
    "Quais são os horários de funcionamento?",
    "Como posso entrar em contato?",
    "Qual é o prazo de entrega?"
]

documentos_simulados = [
    "Documento sobre políticas da empresa",
    "Manual do usuário",
    "FAQ do sistema"
]

print("🧪 TESTANDO SISTEMA RAG MONITORADO")
print("=" * 40)

for i, query in enumerate(queries_teste):
    print(f"\n🔍 Query {i+1}: {query}")
    try:
        resposta = sistema.processar_query(query, documentos_simulados)
        print(f"✅ Processada com sucesso!")
    except Exception as e:
        print(f"❌ Erro: {e}")

print("\n" + "="*50)
metricas_finais = sistema.gerar_dashboard()

## 🌟 Resumão: O que Aprendemos sobre LangSmith

Liiindo! Chegamos ao final do nosso curso completo! 🎉

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-17_img_07.png)

### 🎯 O que o LangSmith faz por você:

1. **🔍 Tracing Completo**: Vê cada passo da sua aplicação
2. **📊 Métricas Detalhadas**: Performance, custo, accuracy
3. **🚨 Monitoramento**: Alertas quando algo vai mal
4. **🧪 Evaluation**: Testa se sua IA está funcionando
5. **🆚 A/B Testing**: Compara diferentes versões
6. **💰 Controle de Custos**: Não quebra o orçamento
7. **🐛 Debugging**: Encontra e corrige problemas

### 📚 Jornada Completa do Curso:
- **Módulos 1-6**: Fundamentos (Chat, Prompts, Chains)
- **Módulos 7-10**: Avançado (Memory, RAG, Vectors)
- **Módulos 11-14**: Aplicações (Agents, Projetos, Deploy)
- **Módulos 15-17**: Evolução (v1.0, LangGraph, LangSmith)

**Dica Final!** LangSmith é essencial para produção. Use sempre!

In [ ]:
# Visualização final: Jornada do curso
modulos = [
    'Intro', 'ChatModel', 'LCEL', 'Prompts', 'Parsers', 
    'Chains', 'Memory', 'Docs', 'Vectors', 'RAG', 
    'Agents', 'Proj1', 'Proj2', 'Deploy', 'v1.0', 
    'LangGraph', 'LangSmith'
]

complexidade = [1, 2, 3, 2, 2, 4, 5, 4, 6, 7, 8, 9, 9, 7, 6, 8, 6]
utilidade = [8, 9, 7, 9, 6, 8, 7, 6, 8, 9, 8, 10, 10, 9, 7, 8, 9]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Gráfico 1: Evolução da complexidade
ax1.plot(range(len(modulos)), complexidade, 'o-', linewidth=3, markersize=8, 
         color='#e74c3c', label='Complexidade')
ax1.fill_between(range(len(modulos)), complexidade, alpha=0.3, color='#e74c3c')
ax1.set_title('Evolução da Complexidade ao Longo do Curso', fontsize=16, pad=20)
ax1.set_ylabel('Nível de Complexidade (1-10)')
ax1.set_xticks(range(len(modulos)))
ax1.set_xticklabels(modulos, rotation=45, ha='right')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Destacar módulos especiais
especiais = [9, 10, 11, 15, 16]  # RAG, Projetos, LangGraph, LangSmith
for idx in especiais:
    ax1.annotate(f'🌟 {modulos[idx]}', 
                xy=(idx, complexidade[idx]), 
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Gráfico 2: Utilidade prática
bars = ax2.bar(range(len(modulos)), utilidade, 
               color=['#2ecc71' if u >= 8 else '#f39c12' if u >= 6 else '#95a5a6' for u in utilidade],
               alpha=0.8, edgecolor='black')

ax2.set_title('Utilidade Prática de Cada Módulo', fontsize=16, pad=20)
ax2.set_ylabel('Utilidade Prática (1-10)')
ax2.set_xticks(range(len(modulos)))
ax2.set_xticklabels(modulos, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bar, util in zip(bars, utilidade):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{util}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("🎓 PARABÉNS! Você completou todo o curso LangChain!")
print("🚀 Agora você está pronto para criar aplicações de IA incríveis!")
print("📊 E o melhor: sabe como monitorar e manter elas funcionando!")

## 🎯 Próximos Passos e Recursos

Tá, mas e agora? Como continuar evoluindo?

### 🔗 Links Importantes:
- **LangSmith**: [smith.langchain.com](https://smith.langchain.com)
- **Documentação**: [docs.smith.langchain.com](https://docs.smith.langchain.com)
- **LangChain**: [python.langchain.com](https://python.langchain.com)
- **Community**: [discord.gg/langchain](https://discord.gg/langchain)

### 🛠️ Projetos para Praticar:
1. **Chatbot com Monitoramento**: Use LangSmith desde o início
2. **Sistema RAG Otimizado**: Compare diferentes estratégias
3. **Agent Multi-Tool**: Monitore performance de cada tool
4. **A/B Testing de Prompts**: Teste melhorias continuamente

### 📚 Continue Aprendendo:
- **LangServe**: Para APIs em produção
- **LangChain Templates**: Projetos prontos
- **Fine-tuning**: Personalize modelos
- **Multi-modal**: Texto + imagem + áudio

**Dica Final!** A IA evolui rápido. Continue estudando e praticando!

---

**Muito obrigado por acompanhar todo o curso!** 🙏

**Pedro Nunes Guth** 🚀

*"A melhor forma de prever o futuro é construí-lo!"*